In [148]:
import pandas as pd
import numpy as np
from utils import load_data
import warnings
warnings.filterwarnings('ignore')

In [149]:
df = pd.DataFrame(load_data('x_train.txt'))
y = pd.DataFrame(load_data('y_train.txt'))


df = pd.concat([df, y], axis=1)

In [150]:
df.columns = [f'col{i}' for i in range(len(df.columns))]

In [141]:
df = df.dropna(axis=1)

In [142]:
def remove_highly_correlated(data, threshold=0.85):
    corr_matrix = data.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return data.drop(columns=to_drop)

In [143]:
def remove_outlier_columns(data, threshold=0.05):
    numeric_data = data.select_dtypes(include=[np.number])
    to_drop = []
    for col in numeric_data.columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((data[col] < (Q1 - 1.5 * IQR)) | (data[col] > (Q3 + 1.5 * IQR))).sum()
        if outliers / data.shape[0] > threshold:
            to_drop.append(col)
    return data.drop(columns=to_drop)

In [144]:
df = remove_highly_correlated(df)

In [145]:
df = remove_outlier_columns(df)

In [146]:
correlation_with_target = df.corr()['col500'].abs()
relevant_features = correlation_with_target[correlation_with_target > 0.05].index.tolist()
df = df[relevant_features]

In [119]:
df.to_csv('step_by_step_5_features.csv', index=False)